In [45]:
import pickle
import pandas as pd
import numpy as np
import scipy.optimize as opt
from scipy.ndimage import label
import matplotlib.pyplot as plt
import pickle

In [46]:

def smooth_daily_variable(df, window_years=10, start_data='1980-01-01', end_data='2010-12-31'):
    """
    Computes the smoothed 90th percentile of daily temperature using a harmonic function.

    Parameters:
    - df: DataFrame with columns ['date', 'temperature']
    - window_years: Number of years for the moving average (default = 10)
    - start_data: Start date for the reference period (default = 1980-01-01)
    - end_data: End date for the reference period (default = 2010-12-31)

    Returns:
    - DataFrame with columns ['date', 'temperature', 'p90_har']
    """

    # Convert date column to datetime
    df["date"] = pd.to_datetime(df["date"])
    df["day_of_year"] = df["date"].dt.dayofyear
    df["year"] = df["date"].dt.year

    # Filter reference period (1980–2010)
    ref_df = df[(df["date"] >= start_data) & (df["date"] <= end_data)].copy()

    # Compute 90th percentile for each day-of-year over a moving 10-year window
    rolling_p90 = (
        ref_df.groupby("day_of_year")["temperature"]
        .rolling(window=window_years, min_periods=1, center=True)
        .quantile(0.90)
        .reset_index(level=0, drop=True)
    )

    # Map smoothed daily percentiles back to the reference DataFrame
    ref_df["daily_p90_smooth"] = rolling_p90

    # Define harmonic function
    def harmonic_func(x, a0, a1, b1, a2, b2, a3, b3):
        return (a0 +
                a1 * np.cos(2 * np.pi * x / 365) + b1 * np.sin(2 * np.pi * x / 365) +
                a2 * np.cos(4 * np.pi * x / 365) + b2 * np.sin(4 * np.pi * x / 365) +
                a3 * np.cos(6 * np.pi * x / 365) + b3 * np.sin(6 * np.pi * x / 365))

    # Fit harmonic function to the smoothed 90th percentile
    x_data = ref_df["day_of_year"].unique()
    y_data = ref_df.groupby("day_of_year")["daily_p90_smooth"].mean().values
    params, _ = opt.curve_fit(harmonic_func, x_data, y_data, maxfev=10000)

    # Apply harmonic smoothing to all dates
    df["p90_har"] = harmonic_func(df["day_of_year"], *params)

    return df[["date", "temperature", "p90_har"]]

def identify_heatwaves(df, min_duration=3):
    """
    Identifies heatwave days as periods where temperature exceeds the harmonic 90th percentile
    for at least 'min_duration' consecutive days.

    Parameters:
    - df: DataFrame with columns ['date', 'temperature', 'p90_har']
    - min_duration: Minimum consecutive days to be considered a heatwave (default = 3)

    Returns:
    - DataFrame with columns ['date', 'temperature', 'p90_har', 'heatwave']
    """

    df["heatwave"] = df["temperature"] > df["p90_har"]

    # Detect consecutive heatwave periods
    heatwave_labels, num_features = label(df["heatwave"].values)

    for i in range(1, num_features + 1):
        heatwave_length = (heatwave_labels == i).sum()
        if heatwave_length < min_duration:
            df.loc[heatwave_labels == i, "heatwave"] = False

    return df[["date", "temperature", "p90_har", "heatwave"]]


def extract_heatwave_events(df):
    """
    Extracts heatwave events from a DataFrame containing identified heatwaves.

    Parameters:
    - df: DataFrame with columns ['date', 'temperature', 'p90_har', 'heatwave']

    Returns:
    - DataFrame with columns ['start_date', 'duration', 'max_temperature']
    """
    heatwave_labels, num_features = label(df["heatwave"].values)
    events = []
    
    for i in range(1, num_features + 1):
        event_dates = df.loc[heatwave_labels == i, "date"]
        event_temps = df.loc[heatwave_labels == i, "temperature"]
        
        start_date = event_dates.min()
        duration = len(event_dates)
        max_temperature = event_temps.max()
        
        events.append([start_date, duration, max_temperature])
    
    return pd.DataFrame(events, columns=["start_date", "duration", "max_temperature"])

In [47]:
filename = '/home/cr2/cmtorres/DATA/KSJ_AWS/KSJ_met_daily_1996_2020.csv'

df = pd.read_csv(filename, sep='\t')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df

,Timestamp,WS(m/s),WDv(deg),Ta(C),RH(%),Prs(hPa),Prcp(mm),Snow(cm),Rsw(W/m2)
0,1996-01-01,4.4,337.5,3.3,91.0,987.0,6.7,0.0,92.9
1,1996-01-02,8.9,312.4,1.5,91.4,985.8,4.5,0.0,113.0
2,1996-01-03,6.3,330.7,2.1,90.1,991.0,0.0,0.0,239.6
3,1996-01-04,6.7,4.1,4.1,88.3,991.1,0.0,0.0,256.1
4,1996-01-05,3.4,95.2,4.1,87.3,981.8,0.0,0.0,86.1
...,...,...,...,...,...,...,...,...,...
8852,2020-03-27,8.4,231.9,-3.9,72.6,972.5,0.1,5.0,88.3
8853,2020-03-28,7.6,290.1,-0.5,77.0,993.7,0.3,0.0,93.1
8854,2020-03-29,7.1,296.3,1.7,90.9,994.4,0.0,0.0,44.2
8855,2020-03-30,7.5,304.3,1.9,92.6,987.9,2.1,0.0,47.0


In [48]:
df_T2 = df[['Timestamp', ' Ta(C)']].rename(columns={'Timestamp':'date', ' Ta(C)':'temperature'})
df_T2

,date,temperature
0,1996-01-01,3.3
1,1996-01-02,1.5
2,1996-01-03,2.1
3,1996-01-04,4.1
4,1996-01-05,4.1
...,...,...
8852,2020-03-27,-3.9
8853,2020-03-28,-0.5
8854,2020-03-29,1.7
8855,2020-03-30,1.9


In [49]:
df_aws1 = smooth_daily_variable(df_T2, start_data='1996-01-01', end_data='2019-12-31')
# Step 2: Identify heatwaves (using 3-day consecutive rule)
HWs_KJS = identify_heatwaves(df_aws1)

/tmp/ipykernel_3673109/4063432384.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["heatwave"] = df["temperature"] > df["p90_har"]


In [50]:
HWs_KJS

,date,temperature,p90_har,heatwave
0,1996-01-01,3.3,2.528010,False
1,1996-01-02,1.5,2.560509,False
2,1996-01-03,2.1,2.592636,False
3,1996-01-04,4.1,2.624360,False
4,1996-01-05,4.1,2.655653,False
...,...,...,...,...
8852,2020-03-27,-3.9,2.369998,False
8853,2020-03-28,-0.5,2.337291,False
8854,2020-03-29,1.7,2.304368,False
8855,2020-03-30,1.9,2.271245,False


In [51]:

def heatwave_labels(df):
    """
    Extracts heatwave events from a DataFrame containing identified heatwaves.

    Parameters:
    - df: DataFrame with columns ['date', 'temperature', 'p90_har', 'heatwave']

    Returns:
    - DataFrame with columns ['start_date', 'duration', 'max_temperature']
    """
    heatwave_labels, num_features = label(df["heatwave"].values)
    df_label = df.copy()
    df_label["label"] = heatwave_labels
    
    return df_label


In [52]:

HWs_lb = (heatwave_labels(HWs_KJS))
HWs_lb['date'] = pd.to_datetime(HWs_lb['date'])  # Asegura que sea datetime
HWs_lb['month'] = HWs_lb['date'].dt.month
HWs_df_fil = HWs_lb[HWs_lb['month'].isin([11, 12, 1, 2, 3])].copy()
HWs_df_fil.drop(columns='month', inplace=True)  # Limpia la columna auxiliar
HWs_df_fil

,date,temperature,p90_har,heatwave,label
0,1996-01-01,3.3,2.528010,False,0
1,1996-01-02,1.5,2.560509,False,0
2,1996-01-03,2.1,2.592636,False,0
3,1996-01-04,4.1,2.624360,False,0
4,1996-01-05,4.1,2.655653,False,0
...,...,...,...,...,...
8852,2020-03-27,-3.9,2.369998,False,0
8853,2020-03-28,-0.5,2.337291,False,0
8854,2020-03-29,1.7,2.304368,False,0
8855,2020-03-30,1.9,2.271245,False,0


In [53]:
# Filtrar eventos AR con 'label' diferente de cero
df_HW1 = HWs_df_fil[HWs_df_fil.label != 0]
df_HW1 = df_HW1[(df_HW1["date"] >= '19960101') & (df_HW1["date"] <= '20061231')]
df_HW1

,date,temperature,p90_har,heatwave,label
46,1996-02-16,4.8,3.260798,True,1
47,1996-02-17,3.7,3.253920,True,1
48,1996-02-18,5.3,3.246030,True,1
49,1996-02-19,5.6,3.237139,True,1
50,1996-02-20,3.4,3.227258,True,1
...,...,...,...,...,...
3738,2006-03-27,4.2,2.402470,True,80
3739,2006-03-28,3.0,2.369998,True,80
3740,2006-03-29,2.6,2.337291,True,80
3741,2006-03-30,3.2,2.304368,True,80


In [54]:
df_HW1.to_csv('data/HWs_KJS_1996_2006_nov_mar.csv', sep='\t', index=False)

In [55]:
df.rename(columns={'Timestamp':'date'}, inplace=True)

In [56]:
HWs_KJS_sel = pd.merge(df_HW1, df, how="left", on="date")

In [57]:
HWs_KJS_sel

,date,temperature,p90_har,heatwave,label,WS(m/s),WDv(deg),Ta(C),RH(%),Prs(hPa),Prcp(mm),Snow(cm),Rsw(W/m2)
0,1996-02-16,4.8,3.260798,True,1,12.1,3.2,4.8,92.8,991.1,22.0,0.0,70.6
1,1996-02-17,3.7,3.253920,True,1,9.9,343.6,3.7,92.6,985.1,2.2,0.0,92.4
2,1996-02-18,5.3,3.246030,True,1,9.6,7.6,5.3,90.1,989.7,9.7,0.0,98.3
3,1996-02-19,5.6,3.237139,True,1,9.8,7.3,5.6,90.5,972.9,4.2,0.0,76.9
4,1996-02-20,3.4,3.227258,True,1,8.3,343.5,3.4,91.0,975.3,0.0,0.0,92.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2006-03-27,4.2,2.402470,True,80,10.8,2.1,4.2,95.5,990.3,2.4,0.0,37.2
123,2006-03-28,3.0,2.369998,True,80,11.9,342.6,3.0,97.6,978.8,3.9,0.0,41.2
124,2006-03-29,2.6,2.337291,True,80,10.8,301.8,2.6,89.0,980.8,0.4,0.0,86.5
125,2006-03-30,3.2,2.304368,True,80,12.8,343.2,3.2,97.4,986.1,1.0,0.0,28.8
